# Create an animation of hexagonal map of Germany, showing mean temperature

In this project, I create a database using **Google Cloud Console** and upload large amount of meteorological data from weather stations around the world. Then I will use this database to filter for relevant information to show the use of postgreSQL. I will do this using Jupyter notebook and **SQLAlchemy** library of Python.

The final goal as an **animated hexagonal map of Germany**, showing for each displayed year the mean temperature in given hexagon.

The data used is available at European Climate Assessment & Dataset as 'Daily mean temperature TG' and has around 7.5 GB.

## Importing libraries and packages

In [ ]:
import os   # to work with files in folders
import pandas as pd   # to work with data frames
from tqdm import tqdm   # to check computation progress

# to communicate with Google Cloud database:
from dotenv import dotenv_values   # to configure the access
from sqlalchemy import create_engine, text   # to use postgreSQL in Python

# for plotting
import plotly.figure_factory as ff   # for hexagonal map
import plotly.graph_objects as go   # for captions
import plotly.io as pio   # to save animation as HTML file

## Concatenating files into one data frame

Define a function to load datasets from files, clean column names, make a data frame with dates and measurements and skip unrelevant columns (quality indicator q_tg = 0).

The data frame contains **the station ID, date and temperature**.

In [ ]:
def parse_file(filename):
    df = pd.read_csv(f'../data/ECA_blend_tg/{filename}', skiprows=19)
    df.columns = df.columns.str.lower().str.strip()
    df['date'] = pd.to_datetime(df['date'], format = '%Y%m%d')
    df = df[df['q_tg'] == 0]   # only relevant observations
    df.pop('souid')
    df.pop('q_tg')
    return df

Show list of files in the directory with data.

In [ ]:
os.listdir('../data/ECA_blend_tg')

Make a new CSV file to append data from all partial data files using the pre-defined function parse_file().

In [ ]:
with open("../data/ECA_blend_tg/mean_temperature.csv", mode="w", newline='') as file:
    for filename in tqdm(os.listdir('../data/ECA_blend_tg')):   # check progress
        if 'TG_STAID' in filename:
            df = parse_file(filename)
            df.to_csv(file, index=False, header=False)

## Creating database

Log into the cloud, 'climate' database using .env file.

In [ ]:
config = dotenv_values('.env')

username = config['USER']
password = config['PASS']
host = config['HOST']
port = config['PORT']

url = f'postgresql://{username}:{password}@{host}:{port}/climate'

engine = create_engine(url, echo=False)

Create table for mean temperatures.

In [ ]:
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS mean_temperature CASCADE;"))
        # CASCADE to drop even if it is bound to other tables by foreign keys
    conn.execute(text("""
        CREATE TABLE mean_temperature (
            staid INT,
            date date,
            tg INT
        );
    """))

Load data into the table.

In [ ]:
df = pd.read_csv('../data/mean_temperature.csv')
df.to_sql('mean_temperature', engine, if_exists='append', index=False)

Analogically, we create **table 'stations'**.

The 'stations' table contains information on **the countries where each station is located and its coordinates**.

In [ ]:
# loading TXT file
df_stations = pd.read_csv('../data/ECA_blend_tg/stations.txt', skiprows = 17)

# cleaning columns
df_stations.columns = df_stations.columns.str.lower().str.strip()

# creating tabnle in database
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS stations CASCADE;"))
    conn.execute(text("""
        CREATE TABLE stations (
            staid INT PRIMARY KEY,
            staname VARCHAR,
            cn VARCHAR(2),
            lat VARCHAR(9),
            lon VARCHAR(10),
            hght INT
        );
    """))
    
# loading data into table
df_stations.to_sql('stations', engine, if_exists='append', index=False)

## Making the animated map

Create a derived table called yearly_mean_temperature that contains the yearly temperature averages for all weather stations from the mean_temperature table.

In [ ]:
with engine.begin() as conn:
    result = conn.execute(text("""
    DROP TABLE IF EXISTS yearly_mean_temperature CASCADE;
    CREATE TABLE yearly_mean_temperature AS
    SELECT staid, AVG(tg) AS yearly_temp, year
    FROM (
        SELECT staid,
            DATE_PART('year', date)::INTEGER AS year,
            tg
        FROM mean_temperature
    ) AS subquery
    GROUP BY staid, year
    ;
    """))

Use JOIN to load yearly data only for German stations. Use only each 5th year for the animation.

In [ ]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT staid, year, yearly_temp, latitude, longitude
    FROM yearly_mean_temperature
    JOIN stations
    USING(staid)
    WHERE cn='DE' AND year BETWEEN 1950 AND 2022 AND (year+3)%5 = 0
    ;
    """))
    rows = result.all()

Create a Pandas data frame from the results.

In [ ]:
df_germany_5y = pd.DataFrame(rows)
df_germany_5y['latitude'] = df_germany_5y['latitude'].astype('float')
df_germany_5y['longitude'] = df_germany_5y['longitude'].astype('float')
df_germany_5y['yearly_temp'] = df_germany_5y['yearly_temp'].astype('float')/10
    # the temperature is given in tenths of degrees of Celsius in the original dataset

Plot the hexagonal animated map.

In [ ]:
# plot and animate
fig_5y = ff.create_hexbin_mapbox(
    data_frame=df_germany_5y,
    lat='latitude',
    lon='longitude',
    opacity=0.9,
    mapbox_style='carto-darkmatter',   # map layout background
    height=650,
    center={'lat': 51.1657, 'lon': 10.4515},   # center of the map view
    color='yearly_temp',
    nx_hexagon=13,   # width of the data in map as number of hexagons
    zoom=4,   # zoom of the map view
    labels={"color": "<i>t</i><sub>avg</sub> (°C)"},
    animation_frame='year',  # animation by year
    title = 'Average temperature in Germany'
)

# dynamic captions:
for frame in fig_5y.frames:
    year = frame['name']
    caption = f'{year}'
    # add the caption as an annotation for each frame:
    frame['layout'].update(
        annotations=[
            go.layout.Annotation(
                text=caption,
                showarrow=False,
                x=0.2,
                y=0.95,
                xanchor='left',
                yanchor='top',
                font=dict(size=25, color="white"),
                bgcolor='black',
                opacity=0.8
            )
        ]
    )

pio.write_html(fig_5y, 'hexbin_map.html')   # save as HTML file
fig_5y.show()   # show in the notebook